In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
path = "/gdrive/My Drive/ML:March2020/data/"
data = pd.read_csv(path+"Chennai_house_univariate_train.csv")
data.head(10)

,Size,Price
0,502.69,6.3
1,503.65,6.3
2,505.46,6.5
3,505.94,6.5
4,507.02,6.6
5,510.77,6.8
6,513.01,7.0
7,517.93,7.2
8,518.84,7.2
9,525.24,7.5


In [4]:
data.shape

(97, 2)

In [5]:
Xcol=data.drop(columns=["Price"])
Ycol=data["Price"]

In [6]:
X=np.array(Xcol)
Y=np.array(Ycol)
Y=np.reshape(Y,(-1,1))
X.shape,Y.shape

((97, 1), (97, 1))

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X, Y, test_size=0.25)

In [8]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((72, 1), (25, 1), (72, 1), (25, 1))

In [9]:
import math
def E_D(point1,point2):
  sum_squared_distance = 0
  for i in range(len(point1)):
    sum_squared_distance += math.pow(point1[i] - point2[i], 2)
  return math.sqrt(sum_squared_distance)

In [10]:
from scipy import stats
import math
def knn(data, input_X, k, distance_fn):
  #create a list to store all predictions 
  predictions=[]   

  #for loop to make predictions for every sample in the input
  for i in range(len(input_X)):
    #store all the distances between the input and the training data
    #along with the index of the training data
    neighbor_distances_and_indices = []

    for index, train in enumerate(data): 
      #find the distance between the new input with every sample in the data
      distance = distance_fn(train[:], input_X[i])   

      #store the distance and the index wrt the training samples   
      neighbor_distances_and_indices.append((distance, index)) 

    #sort the smallest to largest distance
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    
    #pick the first K distances
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k] 
    
    # find the value of y in the training data for the first k distances
    k_nearest_labels = [Y_train[i][0] for distance, i in k_nearest_distances_and_indices]
    
    # find the most common value of y 
    prediction=(stats.mode(k_nearest_labels))

    #append the predicited value to the total_predictions list
    predictions.append(prediction.mode[0])
      
  return predictions # returns the list of all predictions for a given input

In [11]:
clf_query = X_test[:1]
Y_pred = knn(X_train, clf_query, k=5, distance_fn=E_D)
print(Y_pred)

[6.5]


In [12]:
clf_query = X_test
Y_pred = knn(X_train, clf_query, k=5, distance_fn=E_D)

In [14]:
Y_train = Y_train.ravel()

In [17]:
from sklearn.neighbors import KNeighborsRegressor
regressor = KNeighborsRegressor(algorithm = 'brute',n_neighbors=4)
regressor.fit(X_train, Y_train)

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                    weights='uniform')

In [18]:
Y_pred = regressor.predict(X_test)

In [20]:
from sklearn.metrics import r2_score
r2_score(Y_test, Y_pred)

0.984116211728664